# Data Modeling — Overview

## Purpose

Data modeling is the process of creating a visual representation of data structures, relationships, and constraints within an organization's data ecosystem. It serves as the blueprint for how data is stored, organized, and accessed across systems.

**Why Data Modeling Matters:**
- Ensures data consistency and integrity across the organization
- Facilitates communication between business stakeholders and technical teams
- Reduces data redundancy and storage costs
- Improves query performance and system efficiency
- Supports regulatory compliance and data governance

## Key Questions

1. What are the different levels of data modeling and when should each be used?
2. How do you design effective entity-relationship diagrams?
3. When should you use Data Vault vs. traditional dimensional modeling?
4. How does domain-driven design influence data architecture?
5. What trade-offs exist between normalization and denormalization?

---

## 1. Conceptual, Logical, and Physical Data Models

Data modeling follows a **three-tier approach**, progressing from abstract business concepts to concrete implementation details.

### 1.1 Conceptual Data Model

The **conceptual model** is the highest level of abstraction, focusing on business entities and their relationships without technical details.

| Characteristic | Description |
|----------------|-------------|
| **Audience** | Business stakeholders, executives, domain experts |
| **Purpose** | Establish shared understanding of business concepts |
| **Contains** | Entities, relationships, business rules (no attributes) |
| **Technology** | Technology-agnostic |

```
┌──────────────┐         ┌──────────────┐
│   CUSTOMER   │─────────│    ORDER     │
└──────────────┘  places └──────────────┘
                              │
                              │ contains
                              ▼
                         ┌──────────────┐
                         │   PRODUCT    │
                         └──────────────┘
```

### 1.2 Logical Data Model

The **logical model** adds detail to the conceptual model, defining attributes, data types, and keys while remaining database-agnostic.

| Characteristic | Description |
|----------------|-------------|
| **Audience** | Data architects, analysts, developers |
| **Purpose** | Define data structure with business meaning |
| **Contains** | Entities, attributes, primary/foreign keys, relationships |
| **Technology** | Database-agnostic (no vendor-specific features) |

```
┌────────────────────────┐
│       CUSTOMER         │
├────────────────────────┤
│ PK customer_id         │
│    first_name          │
│    last_name           │
│    email               │
│    registration_date   │
└────────────────────────┘
         │
         │ 1:N
         ▼
┌────────────────────────┐
│        ORDER           │
├────────────────────────┤
│ PK order_id            │
│ FK customer_id         │
│    order_date          │
│    total_amount        │
│    status              │
└────────────────────────┘
```

### 1.3 Physical Data Model

The **physical model** is the implementation-ready design, including database-specific optimizations.

| Characteristic | Description |
|----------------|-------------|
| **Audience** | DBAs, developers, operations teams |
| **Purpose** | Optimize for specific database platform |
| **Contains** | Tables, columns, indexes, partitions, constraints |
| **Technology** | Vendor-specific (PostgreSQL, Snowflake, etc.) |

In [ ]:
# Example: Physical Data Model DDL for PostgreSQL

physical_model_ddl = """
-- Customer Table
CREATE TABLE customers (
    customer_id     SERIAL PRIMARY KEY,
    first_name      VARCHAR(100) NOT NULL,
    last_name       VARCHAR(100) NOT NULL,
    email           VARCHAR(255) UNIQUE NOT NULL,
    registration_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    created_at      TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at      TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Order Table with Foreign Key
CREATE TABLE orders (
    order_id        SERIAL PRIMARY KEY,
    customer_id     INTEGER NOT NULL REFERENCES customers(customer_id),
    order_date      TIMESTAMP NOT NULL,
    total_amount    DECIMAL(12,2) NOT NULL,
    status          VARCHAR(20) CHECK (status IN ('pending', 'shipped', 'delivered', 'cancelled')),
    created_at      TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- Performance Indexes
CREATE INDEX idx_orders_customer_id ON orders(customer_id);
CREATE INDEX idx_orders_order_date ON orders(order_date DESC);
CREATE INDEX idx_orders_status ON orders(status) WHERE status = 'pending';
"""

print(physical_model_ddl)

### Model Comparison Summary

| Aspect | Conceptual | Logical | Physical |
|--------|------------|---------|----------|
| **Abstraction Level** | High | Medium | Low |
| **Entities/Tables** | ✓ | ✓ | ✓ |
| **Attributes/Columns** | ✗ | ✓ | ✓ |
| **Data Types** | ✗ | Generic | Specific |
| **Keys** | ✗ | ✓ | ✓ |
| **Indexes** | ✗ | ✗ | ✓ |
| **Partitions** | ✗ | ✗ | ✓ |

---

## 2. ER Diagrams and Relationships

**Entity-Relationship (ER) diagrams** are the standard notation for visualizing data models, showing entities, attributes, and the relationships between them.

### 2.1 Relationship Types (Cardinality)

| Relationship | Notation | Example |
|--------------|----------|----------|
| **One-to-One (1:1)** | `──────` | Person ↔ Passport |
| **One-to-Many (1:N)** | `──────<` | Customer → Orders |
| **Many-to-Many (M:N)** | `>──────<` | Students ↔ Courses |

### 2.2 Crow's Foot Notation

```
Cardinality Symbols:
────────○──  Zero or One (optional)
────────|──  Exactly One (mandatory)
────────<──  Many
────────○<── Zero or Many
────────|<── One or Many

Example: Customer places Orders
┌──────────┐                    ┌──────────┐
│ CUSTOMER │──────|────────○<───│  ORDER   │
└──────────┘                    └──────────┘
  (one)           places        (zero or many)
```

### 2.3 Relationship Patterns

In [ ]:
# Example: Resolving Many-to-Many with Junction Table

junction_table_example = """
-- Many-to-Many: Students ↔ Courses
-- Resolved using junction/bridge table

CREATE TABLE students (
    student_id  SERIAL PRIMARY KEY,
    name        VARCHAR(100) NOT NULL
);

CREATE TABLE courses (
    course_id   SERIAL PRIMARY KEY,
    title       VARCHAR(200) NOT NULL
);

-- Junction Table (Associative Entity)
CREATE TABLE enrollments (
    enrollment_id   SERIAL PRIMARY KEY,
    student_id      INTEGER REFERENCES students(student_id),
    course_id       INTEGER REFERENCES courses(course_id),
    enrollment_date DATE NOT NULL,
    grade           CHAR(2),
    UNIQUE(student_id, course_id)  -- Prevent duplicate enrollments
);

-- Diagram:
-- ┌──────────┐     ┌─────────────┐     ┌──────────┐
-- │ STUDENT  │──|<─│ ENROLLMENT  │─>|──│  COURSE  │
-- └──────────┘     └─────────────┘     └──────────┘
"""

print(junction_table_example)

### 2.4 Self-Referencing Relationships

Entities can have relationships with themselves (recursive relationships).

```
Example: Employee Hierarchy

┌────────────────────────┐
│       EMPLOYEE         │
├────────────────────────┤
│ PK employee_id         │◄────┐
│    name                │     │ reports_to
│ FK manager_id          │─────┘
└────────────────────────┘
```

---

## 3. Data Vault Modeling

**Data Vault** is a hybrid modeling approach designed for enterprise data warehouses, emphasizing **auditability, flexibility, and scalability**.

### 3.1 Core Components

| Component | Purpose | Characteristics |
|-----------|---------|------------------|
| **Hub** | Business keys | Unique identifiers, load metadata |
| **Link** | Relationships | Connects hubs, represents associations |
| **Satellite** | Descriptive data | Attributes, history, temporal data |

### 3.2 Data Vault Architecture

```
                    ┌─────────────────┐
                    │  SAT_CUSTOMER   │
                    │  (attributes)   │
                    └────────┬────────┘
                             │
┌─────────────┐      ┌───────┴───────┐      ┌─────────────┐
│ HUB_CUSTOMER│◄─────│  LINK_ORDER   │─────►│  HUB_ORDER  │
│(business key│      │ (relationship)│      │(business key│
└─────────────┘      └───────────────┘      └──────┬──────┘
                                                   │
                                            ┌──────┴──────┐
                                            │  SAT_ORDER  │
                                            │ (attributes)│
                                            └─────────────┘
```

### 3.3 Data Vault vs. Dimensional Modeling

| Aspect | Data Vault | Dimensional (Star Schema) |
|--------|------------|---------------------------|
| **Primary Use** | Raw data storage, EDW | Reporting, analytics |
| **Flexibility** | High (additive) | Medium (requires redesign) |
| **History** | Built-in versioning | Slowly Changing Dimensions |
| **Complexity** | Higher learning curve | Easier to understand |
| **Load Pattern** | Insert-only | Insert/Update |
| **Query Performance** | Requires Business Vault | Optimized for BI |

In [ ]:
# Example: Data Vault Table Structures

data_vault_ddl = """
-- HUB: Business Keys
CREATE TABLE hub_customer (
    hub_customer_hk     CHAR(32) PRIMARY KEY,  -- Hash key
    customer_bk         VARCHAR(50) NOT NULL,   -- Business key
    load_date           TIMESTAMP NOT NULL,
    record_source       VARCHAR(100) NOT NULL
);

-- SATELLITE: Descriptive Attributes (with history)
CREATE TABLE sat_customer (
    hub_customer_hk     CHAR(32) NOT NULL,
    load_date           TIMESTAMP NOT NULL,
    load_end_date       TIMESTAMP,              -- NULL = current record
    record_source       VARCHAR(100) NOT NULL,
    hash_diff           CHAR(32) NOT NULL,      -- Detect changes
    -- Attributes
    first_name          VARCHAR(100),
    last_name           VARCHAR(100),
    email               VARCHAR(255),
    PRIMARY KEY (hub_customer_hk, load_date)
);

-- LINK: Relationships
CREATE TABLE link_customer_order (
    link_customer_order_hk  CHAR(32) PRIMARY KEY,
    hub_customer_hk         CHAR(32) NOT NULL,
    hub_order_hk            CHAR(32) NOT NULL,
    load_date               TIMESTAMP NOT NULL,
    record_source           VARCHAR(100) NOT NULL
);
"""

print(data_vault_ddl)

---

## 4. Data Mesh and Domain-Driven Design

**Data Mesh** is a decentralized data architecture paradigm that applies domain-driven design principles to data management.

### 4.1 Core Principles

| Principle | Description |
|-----------|-------------|
| **Domain Ownership** | Data is owned by domain teams, not a central data team |
| **Data as a Product** | Treat data as a product with SLAs, documentation, quality |
| **Self-Serve Platform** | Infrastructure enables autonomous domain teams |
| **Federated Governance** | Global standards with domain autonomy |

### 4.2 Domain-Driven Data Modeling

```
┌─────────────────────────────────────────────────────────────────┐
│                     ORGANIZATION                                │
├─────────────────┬─────────────────┬─────────────────────────────┤
│  SALES DOMAIN   │  ORDER DOMAIN   │     INVENTORY DOMAIN        │
├─────────────────┼─────────────────┼─────────────────────────────┤
│ • Customer      │ • Order         │ • Product                   │
│ • Lead          │ • OrderLine     │ • Warehouse                 │
│ • Opportunity   │ • Payment       │ • StockLevel                │
│                 │                 │                             │
│ [Sales Team     │ [Fulfillment    │ [Supply Chain               │
│  Owns Data]     │  Team Owns Data]│  Team Owns Data]            │
└─────────────────┴─────────────────┴─────────────────────────────┘
```

### 4.3 Bounded Contexts in Data Modeling

Each domain has its own **bounded context** with potentially different representations of shared concepts.

In [ ]:
# Example: Same Entity, Different Bounded Contexts

bounded_contexts = {
    "sales_context": {
        "entity": "Customer",
        "attributes": [
            "customer_id",
            "name",
            "email",
            "lead_source",
            "sales_rep_id",
            "lifetime_value",
            "segment"
        ],
        "focus": "Revenue and relationship management"
    },
    "fulfillment_context": {
        "entity": "Customer",
        "attributes": [
            "customer_id",
            "name",
            "shipping_address",
            "delivery_preferences",
            "phone_number"
        ],
        "focus": "Order delivery and logistics"
    },
    "support_context": {
        "entity": "Customer",
        "attributes": [
            "customer_id",
            "name",
            "email",
            "support_tier",
            "ticket_history",
            "preferred_contact_method"
        ],
        "focus": "Customer service and issue resolution"
    }
}

for context, details in bounded_contexts.items():
    print(f"\n{context.upper().replace('_', ' ')}")
    print(f"  Entity: {details['entity']}")
    print(f"  Focus: {details['focus']}")
    print(f"  Attributes: {', '.join(details['attributes'])}")

### 4.4 Data Products

In Data Mesh, each domain exposes **data products** with clear contracts:

| Data Product Attribute | Description |
|------------------------|-------------|
| **Discoverability** | Registered in data catalog |
| **Addressability** | Unique, stable access path |
| **Trustworthiness** | Quality metrics, SLAs |
| **Self-describing** | Schema, metadata, documentation |
| **Interoperability** | Standard formats, APIs |
| **Security** | Access controls, compliance |

```yaml
# Example: Data Product Contract
data_product:
  name: customer_360
  domain: sales
  owner: sales-data-team@company.com
  
  schema:
    format: parquet
    location: s3://sales-domain/products/customer_360/
    
  quality:
    freshness: "< 1 hour"
    completeness: "> 99%"
    accuracy: "> 99.5%"
    
  access:
    classification: confidential
    allowed_consumers:
      - marketing-domain
      - analytics-team
```

---

## 5. Normalization vs. Denormalization

### 5.1 Normal Forms

| Normal Form | Rule | Purpose |
|-------------|------|----------|
| **1NF** | Atomic values, no repeating groups | Eliminate redundancy |
| **2NF** | 1NF + No partial dependencies | Remove partial key dependencies |
| **3NF** | 2NF + No transitive dependencies | Remove non-key dependencies |
| **BCNF** | Every determinant is a candidate key | Stricter 3NF |

### 5.2 When to Normalize vs. Denormalize

| Scenario | Normalize | Denormalize |
|----------|-----------|-------------|
| **OLTP Systems** | ✓ | |
| **Data Warehouses** | | ✓ |
| **Write-heavy workloads** | ✓ | |
| **Read-heavy analytics** | | ✓ |
| **Data integrity critical** | ✓ | |
| **Query performance critical** | | ✓ |

---

## 📌 Takeaways

### Key Concepts

1. **Three-Tier Modeling**: Progress from conceptual → logical → physical models to bridge business and technical requirements

2. **ER Diagrams**: Use proper cardinality notation (Crow's Foot) and resolve M:N relationships with junction tables

3. **Data Vault**: Hub-Link-Satellite pattern for auditable, flexible enterprise data warehouses

4. **Data Mesh**: Decentralized ownership with domain teams treating data as products

5. **Normalization Trade-offs**: Balance data integrity (normalize) vs. query performance (denormalize)

### Best Practices

| Practice | Benefit |
|----------|----------|
| Start with conceptual model | Align business understanding |
| Document relationships clearly | Reduce ambiguity |
| Use consistent naming conventions | Improve maintainability |
| Version control your models | Track evolution |
| Validate against use cases | Ensure fitness for purpose |

### Further Reading

- *The Data Warehouse Toolkit* — Ralph Kimball
- *Building a Scalable Data Warehouse with Data Vault 2.0* — Dan Linstedt
- *Data Mesh* — Zhamak Dehghani
- *Domain-Driven Design* — Eric Evans